In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample

# Preparazione dati e filtraggio match International, International Gold, Masters, Masters Cup

In [ ]:
# Caricamento dati
df = pd.read_csv('./atp_tennis.csv')

# Salva il numero originale di righe
original_rows = len(df)

# Filtra ranking e punti negativi o nulli
rows_before = len(df)
df = df[
    (df['Rank_1'] > 0) & 
    (df['Rank_2'] > 0) & 
    (df['Pts_1'] > 0) & 
    (df['Pts_2'] > 0)
]
rows_after = len(df)
print(f"\nRighe rimosse per ranking o punti non validi: {rows_before - rows_after}")

# Mostra distribuzione originale delle categorie
print("\nDistribuzione originale delle categorie dei tornei:")
print(df['Series'].value_counts())
print("\nDistribuzione originale dei Round:")
print(df['Round'].value_counts())

# Filtra le categorie non rilevanti dal target
rows_before_series = len(df)
df = df[~df['Series'].isin(['International', 'International Gold', 'Masters', 'Masters Cup'])]
rows_after_series = len(df)

print(f"\nRighe rimosse dal filtraggio delle Series: {rows_before_series - rows_after_series}")
print("Categorie rimosse:")
for category in ['International', 'International Gold', 'Masters', 'Masters Cup']:
    count = len(df[df['Series'] == category])
    print(f"- {category}: {count} partite")

# Mostra distribuzione delle categorie dopo il filtraggio delle Series
print("\nDistribuzione delle categorie dei tornei dopo il filtraggio delle Series:")
print(df['Series'].value_counts())

# Filtra Round Robin
rows_before_round = len(df)
df = df[df['Round'] != 'Round Robin']
rows_after_round = len(df)

print(f"\nRighe rimosse dal filtraggio del Round: {rows_before_round - rows_after_round}")
print(f"- Round Robin: {rows_before_round - rows_after_round} partite")

# Mostra distribuzione dei Round dopo il filtraggio
print("\nDistribuzione dei Round dopo il filtraggio del Round Robin:")
print(df['Round'].value_counts())

# Riepilogo finale
print("\nRiepilogo del filtraggio:")
print(f"Righe originali: {original_rows}")
print(f"Righe dopo filtraggio Series: {rows_after_series}")
print(f"Righe dopo filtraggio Round: {rows_after_round}")
print(f"Totale righe rimosse: {original_rows - len(df)}")
print(f"Percentuale dati mantenuti: {(len(df)/original_rows)*100:.2f}%")

# Feature basata su Best_of e Grand Slam
df['is_best_of_5'] = (df['Best of'] == 5).astype(int)
df['is_grand_slam'] = df['is_best_of_5']

# Feature basate sul Winner e ranking
df['winner_rank'] = df.apply(lambda x: x['Rank_1'] if x['Winner'] == 1 else x['Rank_2'], axis=1)
df['rank_diff'] = abs(df['Rank_1'] - df['Rank_2'])
df['avg_rank'] = (df['Rank_1'] + df['Rank_2']) / 2

# Feature basate sul Round
round_mapping = {
    '1st Round': 1,
    '2nd Round': 2,
    '3rd Round': 3,
    '4th Round': 4,
    'Quarterfinals': 5,
    'Semifinals': 6,
    'The Final': 7
}

# Converti il Round in numero e calcola dimensione tabellone
df['round_number'] = df['Round'].map(round_mapping)
df['has_3rd_round'] = (df['Round'].isin(['3rd Round', '4th Round'])).astype(int)
# Calcola dimensione tabellone con controllo sul tipo di torneo
df['draw_size'] = df.apply(lambda x: 
    128 if x['Round'] == '4th Round' else  # Grand Slam e alcuni Masters 1000
    96 if x['has_3rd_round'] == 1 and x['Series'] == 'Masters 1000' else  # Solo Masters 1000
    64 if x['Round'] == '3rd Round' else    # ATP 500
    32,                                     # ATP 250
    axis=1)

# Bilanciamento dataset

In [ ]:
# Separa il dataset per categoria
df_250 = df[df['Series'] == 'ATP250']
df_500 = df[df['Series'] == 'ATP500']
df_1000 = df[df['Series'] == 'Masters 1000']
df_slam = df[df['Series'] == 'Grand Slam']

# Determina la dimensione target (usiamo la categoria con meno esempi)
n_samples = min(len(df_500), len(df_1000), len(df_slam))

# Downsampling della classe maggioritaria (ATP250)
df_250_balanced = resample(df_250,
                         replace=False,
                         n_samples=n_samples,
                         random_state=42)

# Mantieni le altre classi invariate se hanno già circa n_samples elementi
# altrimenti fai upsampling
df_500_balanced = resample(df_500,
                         replace=True,
                         n_samples=n_samples,
                         random_state=42)

df_1000_balanced = resample(df_1000,
                          replace=True,
                          n_samples=n_samples,
                          random_state=42)

df_slam_balanced = resample(df_slam,
                          replace=True,
                          n_samples=n_samples,
                          random_state=42)

# Combina i dataset bilanciati
df_balanced = pd.concat([df_250_balanced, 
                        df_500_balanced,
                        df_1000_balanced,
                        df_slam_balanced])

# Controlla la distribuzione delle classi nel dataset bilanciato
print("Distribuzione delle classi nel dataset bilanciato:")
print(df_balanced['Series'].value_counts())

In [ ]:
# Calcola il ranking medio per categoria del torneo
ranking_medio = df_balanced.groupby('Series')['avg_rank'].mean()

# Stampa i risultati
print("Ranking medio per categoria del torneo:")
print(ranking_medio)

# Preparazione features per classificazione

In [ ]:
# Preparazione features iniziali
features = ['Round']  # Solo Round come feature categorica

# Codifica variabili categoriche
le = LabelEncoder()
df_balanced['Round'] = le.fit_transform(df_balanced['Round'])
print(f"\nCategorie per Round: {le.classes_}")

# Codifica della variabile target
target = 'Series'
df_balanced['Series_encoded'] = le.fit_transform(df_balanced[target])
print(f"\nCategorie per Series: {le.classes_}")

# Aggiorna la lista delle features, organizzate per importanza e tipologia
features.extend([
    # Feature strutturali del torneo   
    'draw_size',
    'round_number',
    
    # Feature di ranking
    'Rank_1', 'Rank_2',
    'rank_diff',
    'winner_rank',
    'is_best_of_5',
    
    # Feature di punti
    'Pts_1', 'Pts_2'
])

# Preparazione dati usando il dataset bilanciato
X = df_balanced[features].copy()
y = le.fit_transform(df_balanced[target])

print(X.head(20))

# Standardizzazione delle feature numeriche
numeric_features = [col for col in features if col != 'Round']

# Converti le colonne numeriche in float64 prima della standardizzazione
for feature in numeric_features:
    X[feature] = X[feature].astype('float64')

# Applica la standardizzazione
scaler = StandardScaler()
X.loc[:, numeric_features] = scaler.fit_transform(X[numeric_features])

# Split dei dati
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Controlla la distribuzione delle classi nel dataset finale
print("\nDistribuzione delle classi nel dataset finale:")
print(pd.Series(y).value_counts())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calcolo della matrice di correlazione
correlation_matrix = X.corr()

# Plot della heatmap della matrice di correlazione
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", vmin=-1, vmax=1)
plt.title("Heatmap della matrice di correlazione tra le feature")
plt.show()

# Addestramento e valutazione modelli

In [ ]:

# Dizionario dei classificatori
classifiers = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Valutazione dei modelli
results = {}

# Dizionario dei classificatori
classifiers = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(
        kernel='rbf',  # Cambia il kernel
        C=10.0,        # Aumenta C per ridurre la regolarizzazione
        gamma='scale', 
        random_state=42
    ),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Assicurati che i dati siano scalati correttamente prima del ciclo
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Valutazione dei modelli
results = {}

for name, clf in classifiers.items():
    print(f"\n{'-'*50}")
    print(f"Valutazione del classificatore: {name}")
    
    # Addestramento
    if name == 'SVM':
        # Usa i dati scalati per SVM
        clf.fit(X_train_scaled, y_train)
        y_pred = clf.predict(X_test_scaled)
    else:
        # Per gli altri classificatori usa i dati originali
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
    
    # Salvataggio risultati
    results[name] = {
        'classification_report': classification_report(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }
    
    # Report di classificazione
    print("\nClassification Report:")
    print(results[name]['classification_report'])
    
    # Matrice di confusione
    plt.figure(figsize=(10, 8))
    cm = results[name]['confusion_matrix']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=le.classes_,
                yticklabels=le.classes_)
    plt.title(f'Matrice di Confusione - {name}')
    plt.ylabel('Valore Reale')
    plt.xlabel('Valore Predetto')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Feature importance per Random Forest e Gradient Boosting
    if name in ['Random Forest', 'Gradient Boosting']:
        feature_importance = pd.DataFrame({
            'feature': features,
            'importance': clf.feature_importances_
        })
        feature_importance = feature_importance.sort_values('importance', ascending=False)
        
        plt.figure(figsize=(10, 6))
        sns.barplot(x='importance', y='feature', data=feature_importance)
        plt.title(f'Importanza delle Feature - {name}')
        plt.xlabel('Importanza')
        plt.ylabel('Feature')
        plt.tight_layout()
        plt.show()
        
        print("\nImportanza delle feature in percentuale:")
        for idx, row in feature_importance.iterrows():
            print(f"{row['feature']}: {row['importance']*100:.2f}%")



In [ ]:
# Confronto delle performance
performances = []
for name, result in results.items():
    report = classification_report(y_test, classifiers[name].predict(X_test), output_dict=True)
    performances.append({
        'Classificatore': name,
        'Accuracy': report['accuracy'],
        'Macro F1-Score': report['macro avg']['f1-score']
    })

# Visualizzazione confronto performance
performance_df = pd.DataFrame(performances)
plt.figure(figsize=(12, 6))
performance_df.plot(x='Classificatore', y=['Accuracy', 'Macro F1-Score'], kind='bar')
plt.title('Confronto Performance Classificatori')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nTabella delle Performance:")
print(performance_df.to_string(index=False))